# Final Tutorial Notebook
#### Emily Gong, Robert Morrison

In this notebook we will explore the transcripts from UN General Debates. Our goal is to understand what differences exist between speakers of different language backgrounds

In [1]:
import numpy as np
import pandas as pd

import re

import requests
import bs4

import nltk
import spacy
from spacy import displacy #display word entities
import en_core_web_sm #language model
from collections import Counter
nlp = en_core_web_sm.load() 

# Below are libraries for LDA using gensim which is provides less control 
from nltk.corpus import stopwords #stop words to be filited out
from gensim import models, corpora #Used for LDA topic modeling
from sklearn.metrics.pairwise import euclidean_distances 
import pyLDAvis.gensim #python library for interactive topic model visualization
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#Below are libraries for LDA using sklearn


from sklearn.feature_extraction.text import TfidfVectorizer

# Introduction

The United Nations is an international organization of independent states to promote peace and international cooperation and security. Currently, there are 193 soverign states and they meet every year in regular session on the Tuesday of the third week in September. 

# Loading Data
## Raw Data

In [2]:
data = pd.read_csv("un-general-debates.csv")
data.head()

,session,year,country,text
0,44,1989,MDV,﻿It is indeed a pleasure for me and the member...
1,44,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ..."
2,44,1989,NER,"﻿\nMr. President, it is a particular pleasure ..."
3,44,1989,URY,﻿\nDuring the debate at the fortieth session o...
4,44,1989,ZWE,﻿I should like at the outset to express my del...


In [3]:
print(len(data))

7507


In [4]:
countries = data["country"].unique()
print(countries)
print(len(countries))

['MDV' 'FIN' 'NER' 'URY' 'ZWE' 'PHL' 'SDN' 'RUS' 'CHN' 'ESP' 'SUR' 'ARG'
 'SLV' 'MYS' 'NPL' 'PRT' 'COL' 'BLR' 'MAR' 'LCA' 'EGY' 'MEX' 'BEL' 'BRN'
 'RWA' 'CAN' 'ALB' 'GRC' 'KNA' 'GUY' 'LBR' 'ATG' 'MOZ' 'JPN' 'YDYE' 'GAB'
 'BGD' 'SWE' 'TUR' 'TCD' 'SYR' 'CMR' 'JAM' 'LUX' 'ITA' 'AGO' 'CRI' 'CSK'
 'BFA' 'MNG' 'BHR' 'HTI' 'OMN' 'CIV' 'TGO' 'CYP' 'MUS' 'MMR' 'ARE' 'GTM'
 'GRD' 'LBY' 'LKA' 'TZA' 'SGP' 'NOR' 'LAO' 'ISL' 'AFG' 'CHL' 'DMA' 'UKR'
 'KEN' 'BLZ' 'FRA' 'MLI' 'VCT' 'VEN' 'MLT' 'GHA' 'GIN' 'GBR' 'ISR' 'YUG'
 'BRB' 'IRQ' 'HUN' 'AUT' 'POL' 'GNB' 'BWA' 'MRT' 'SWZ' 'DNK' 'DOM' 'MDG'
 'NIC' 'BDI' 'CUB' 'IRN' 'PAK' 'SEN' 'BGR' 'YEM' 'STP' 'NLD' 'VUT' 'BOL'
 'PNG' 'SLB' 'DEU' 'ROU' 'KHM' 'TUN' 'BRA' 'IND' 'IDN' 'AUS' 'COD' 'HND'
 'GNQ' 'FJI' 'IRL' 'DZA' 'USA' 'LSO' 'GMB' 'PER' 'DDR' 'THA' 'JOR' 'COG'
 'NGA' 'ECU' 'SAU' 'QAT' 'SYC' 'ETH' 'TTO' 'PRY' 'VNM' 'NZL' 'PAN' 'MWI'
 'DJI' 'BEN' 'SOM' 'ZMB' 'CPV' 'BHS' 'KWT' 'UGA' 'COM' 'ZAF' 'LBN' 'SLE'
 'KOR' 'BIH' 'TON' 'EU' 'HRV' 'NRU' 'TUV' 'NAM' 'S

In [5]:
years = data["year"].unique()
print(years)
print(len(years))
# 46 Examples of each country

[1989 1970 2013 1985 2008 1991 1986 2002 1975 1996 2012 1997 1978 1988
 2010 1984 1995 2009 1971 1976 1983 1979 1999 2005 1987 1982 1998 2003
 2004 1980 2014 2011 1974 2015 1993 1977 1981 2000 1992 1990 1973 1994
 1972 2006 2007 2001]
46


## Scraping

In [6]:
# Getting a decoding table and access to each countries Wikipedia page
url = "https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3"
r = requests.get(url)
soup = bs4.BeautifulSoup(r.text)

In [7]:
found = soup.find("div", class_="plainlist").find("ul")

In [8]:
lookup = {} # dict {country code : [Country Name, wikipedia link]}

for child in found.children:
    if isinstance(child, bs4.Tag):
        c_code = child.find("span").text
        link = child.find("a").get("href")
        country = child.find("a").text
        
        lookup[c_code] = [country, link]

lookup["CSK"] = ["Czechoslovakia", "/wiki/Czechoslovakia"]
lookup["YDYE"] = ["South Yemen", "/wiki/South_Yemen"]
lookup["YUG"] = ["Yugoslavia", "/wiki/Yugoslavia"]
lookup["DDR"] = ["East Germany", "/wiki/East_Germany"]
lookup["EU"] = ["European Union", "/wiki/European_Union"]

In [9]:
data["name"] = [lookup[code][0] for code in data["country"]]

# EDA

In [10]:
sample_text = data.loc[0]["text"]
sample_text = sample_text.replace(u'\ufeff', '')
print(sample_text[0:100])

It is indeed a pleasure for me and the members of my delegation to extend to Ambassador Garba our si


In [40]:
# nltk.word_tokenize(sample_text)
tokens = nltk.word_tokenize(sample_text.lower())
print(tokens[0:10])

['it', 'is', 'indeed', 'a', 'pleasure', 'for', 'me', 'and', 'the', 'members']


In [41]:
pos = nltk.pos_tag(tokens)
pos[:10]

[('it', 'PRP'),
 ('is', 'VBZ'),
 ('indeed', 'RB'),
 ('a', 'DT'),
 ('pleasure', 'NN'),
 ('for', 'IN'),
 ('me', 'PRP'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('members', 'NNS')]

In [42]:
# Don't use this bruh
chunk = nltk.ne_chunk(pos)
chunk[:20]

[('it', 'PRP'),
 ('is', 'VBZ'),
 ('indeed', 'RB'),
 ('a', 'DT'),
 ('pleasure', 'NN'),
 ('for', 'IN'),
 ('me', 'PRP'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('members', 'NNS'),
 ('of', 'IN'),
 ('my', 'PRP$'),
 ('delegation', 'NN'),
 ('to', 'TO'),
 ('extend', 'VB'),
 ('to', 'TO'),
 ('ambassador', 'VB'),
 ('garba', 'VB'),
 ('our', 'PRP$'),
 ('sincere', 'JJ')]

In [43]:
nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm')
doc = nlp(sample_text)
print(len(doc.ents))

print([(X.text, X.label_) for X in doc.ents[:20]])

labels = [x.label_ for x in doc.ents]
Counter(labels)

items= [x.text for x in doc.ents]
Counter(items).most_common(3)

sentences = [x for x in doc.sents]
print(sentences[21])

displacy.render(nlp(str(sentences[21])), jupyter=True, style='ent')

142
[('the General Assembly', 'ORG'), ('Assembly', 'ORG'), ('the past year', 'DATE'), ('\n', 'GPE'), ('Dante Caputo', 'PERSON'), ('forty-third', 'CARDINAL'), ('the General Assembly', 'ORG'), ('\n', 'GPE'), ('As in previous years', 'TIME'), ('the United Nations', 'ORG'), ('the Charter of the United Nations', 'LAW'), ('Organization', 'ORG'), ('recent years', 'DATE'), ('Organization', 'ORG'), ('Today', 'DATE'), ('Recent years', 'DATE'), ('-Power', 'ORG'), ('\n', 'GPE'), ('Africa', 'LOC'), ('Namibia', 'GPE')]
The 1980s have witnessed one of the longest spells of growth for the industrialized countries, while the situation in the South, particularly in the least developed countries, continues to deteriorate.


In [44]:
vect = TfidfVectorizer()

tfidf = vect.fit_transform(data["text"])

In [45]:
tfidf

<7507x54892 sparse matrix of type '<class 'numpy.float64'>'
	with 6900203 stored elements in Compressed Sparse Row format>

In [46]:
# LDA Topic Modeling

In [47]:
num_topic = 2
stop_words = stopwords.words('english')
gensim_tokenized_data = []
# gensim needs the data to have removed stop words 
# Later: can create a method to produce clean data
gensim_tokenized_data.append([t.lower() for t in tokens if t not in stop_words and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)])
gensim_dictionary = corpora.Dictionary(gensim_tokenized_data)
# Convert document into the bag-of-words (BoW) format = list of (token_id, token_count) tuples.
gensim_corpus = [gensim_dictionary.doc2bow(text) for text in gensim_tokenized_data]
lda_model = models.LdaModel(corpus=gensim_corpus, num_topics=num_topic, id2word=gensim_dictionary)

In [48]:
# print("LDA Model:")
 
# for idx in range(num_topic):
#     # Print the first 10 most representative topics
#     print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
# print("=" * 20)

In [49]:
def most_similar(x, Z, top_n=5):
    dists = euclidean_distances(x.reshape(1, -1), Z)
    pairs = enumerate(dists[0])
    most_similar = sorted(pairs, key=lambda item: item[1])[:top_n]
    return most_similar


In [50]:
pyLDAvis.gensim.prepare(lda_model, gensim_corpus, gensim_dictionary)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.001566  0.0       1        1  75.662292
1     -0.001566  0.0       2        1  24.337711, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
157   Default   5.000000      countries   5.000000  30.0000  30.0000
735   Default   8.000000      situation   8.000000  29.0000  29.0000
736   Default   5.000000          small   5.000000  28.0000  28.0000
233   Default   8.000000        efforts   8.000000  27.0000  27.0000
145   Default   6.000000      continues   6.000000  26.0000  26.0000
412   Default   9.000000  international   9.000000  25.0000  25.0000
559   Default   7.000000          peace   7.000000  24.0000  24.0000
422   Default   4.000000          issue   4.000000  23.0000  23.0000
874   Default   6.000000        welcome   6.000000  22.0000  22.0000
584   Default   6.000000      political   6.000000  21.0000  21.0000
757   Default   2.000000          state   2.000000  20.0000  20.0000
787   Default   4.000000          taken   4.000000  19.0000  19.0000
461   Default   5.000000       maldives   5.000000  18.0000  18.0000
706   Default  11.000000       security  11.000000  17.0000  17.0000
519   Default   6.000000        nuclear   6.000000  16.0000  16.0000
130   Default   5.000000       conflict   5.000000  15.0000  15.0000
834   Default   4.000000         united   4.000000  14.0000  14.0000
562   Default   4.000000         people   4.000000  13.0000  13.0000
503   Default   3.000000       national   3.000000  12.0000  12.0000
560   Default   2.000000       peaceful   2.000000  11.0000  11.0000
893   Default   2.000000          zones   2.000000  10.0000  10.0000
643   Default   3.000000         recent   3.000000   9.0000   9.0000
758   Default   9.000000         states   9.000000   8.0000   8.0000
872   Default   4.000000        weapons   4.000000   7.0000   7.0000
784   Default   3.000000         system   3.000000   6.0000   6.0000
554   Default   3.000000        parties   3.000000   5.0000   5.0000
184   Default   4.000000     delegation   4.000000   4.0000   4.0000
115   Default   3.000000      community   3.000000   3.0000   3.0000
890   Default   3.000000          years   3.000000   2.0000   2.0000
229   Default   6.000000       economic   6.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
546    Topic2   0.695030          ozone   2.385580   0.1799  -6.2928
735    Topic2   2.317061      situation   8.566844   0.1055  -5.0887
445    Topic2   0.858658        lebanon   3.001027   0.1618  -6.0814
233    Topic2   2.252885        efforts   8.687546   0.0635  -5.1168
145    Topic2   1.637977      continues   6.151273   0.0899  -5.4355
643    Topic2   1.008570         recent   3.642270   0.1291  -5.9205
562    Topic2   1.158477         people   4.283524   0.1055  -5.7819
461    Topic2   1.426858       maldives   5.625147   0.0414  -5.5735
874    Topic2   1.695366        welcome   6.966534  -0.0001  -5.4011
584    Topic2   1.689760      political   6.977075  -0.0049  -5.4044
559    Topic2   1.815653          peace   7.663493  -0.0269  -5.3325
834    Topic2   1.271587         united   4.993982   0.0452  -5.6887
784    Topic2   0.981961         system   3.692317   0.0887  -5.9472
554    Topic2   0.979205        parties   3.697499   0.0845  -5.9500
412    Topic2   2.145650  international   9.812428  -0.1071  -5.1655
130    Topic2   1.386414       conflict   5.701213  -0.0008  -5.6023
519    Topic2   1.501592        nuclear   6.407783  -0.0378  -5.5225
115    Topic2   0.969706      community   3.715365   0.0699  -5.9598
706    Topic2   2.222450       security  11.514374  -0.2318  -5.1304
872    Topic2   1.094691        weapons   4.403490   0.0212  -5.8385
890    Topic2   0.968771          years   3.717123   0.0685  -5.9607
758    Topic2   1.806332         st

The left panel, labeld Intertopic Distance Map, circles represent different topics and the distance between them. Similar topics appear closer and the dissimilar topics farther. The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus. An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.


The right panel, include the bar chart of the top 30 terms. When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics. Selecting each topic on the right, modifies the bar chart to show the "relevant" terms for the selected topic. Relevence is defined as in footer 2 and can be tuned by parameter  λ
 , smaller  λ
  gives higher weight to the term's distinctiveness while larger  λ
 s corresponds to probablity of the term occurance per topics.

Therefore, to get a better sense of terms per topic we'll use  λ
 =0.


In [35]:
vect.stop_words_

set()